In [9]:
import pandas as pd 
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from tqdm import tqdm

In [10]:
mappings = [
    ((405, 408), 'occ3_clean'),
    ((415, 427), 'occ3_protect'),
    ((415, 415), 'occ3_guard'),
    ((425, 427), 'occ3_guard'),
    ((433, 444), 'occ3_food'),
    ((445, 447), 'occ3_shealth'),
    ((448, 455), 'occ3_janitor'),
    ((457, 458), 'occ3_beauty'),
    ((459, 467), 'occ3_recreation'),
    ((468, 468), 'occ3_child'),
    ((469, 472), 'occ3_others'),
    ((3, 22), 'occ2_exec'),
    ((23, 37), 'occ2_mgmtrel'),
    ((43, 200), 'occ2_prof'),
    ((203, 235), 'occ2_tech'),
    ((243, 258), 'occ2_finsales'),
    ((274, 283), 'occ2_retsales'),
    ((303, 389), 'occ2_cleric'),
    ((417, 423), 'occ2_firepol'),
    ((473, 475), 'occ2_farmer'),
    ((479, 498), 'occ2_otheragr'),
    ((503, 549), 'occ2_mechanic'),
    ((558, 599), 'occ2_constr'),
    ((614, 617), 'occ2_mining'),
    ((628, 699), 'occ2_product'),
    ((703, 799), 'occ2_operator'),
    ((803, 889), 'occ2_transp'),
]

In [11]:
def map_occupations(occ1990dd):
    for (start, end), occupation in mappings:
        if start <= occ1990dd <= end:
            return occupation
    return 'other'

def expand_ranges(df):
    expanded_rows = []
    for index, row in df.iterrows():
        if '-' in index:
            start, end = map(int, index.split('-'))
            for i in range(start, end + 1):
                expanded_rows.append((str(i), row['Name']))
        else:
            expanded_rows.append((index, row['Name']))
    return pd.DataFrame(expanded_rows, columns=['Sector', 'Name']).set_index('Sector')

In [12]:
df = dd.read_csv('raw/usa_00015.csv', assume_missing=True, dtype={'INDNAICS': 'string'})
df_cz = pd.read_csv('raw/cz_mappings.csv')
df_cz = df_cz[['LMA/CZ', 'FIPS']]

df['FIPS'] = df['STATEFIP'] * 1000 + df['COUNTYFIP']
df['COMZONE'] = df['FIPS'].map(df_cz.set_index('FIPS')['LMA/CZ'])

with ProgressBar():
    df.shape[0].compute()

# with ProgressBar():
#     df = df.compute(scheduler='threads')

/home/sjone9/miniconda3/lib/python3.12/site-packages/dask_expr/_collection.py:4160: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('FIPS', 'float64'))

  warnings.warn(meta_warning(meta))


122284061

In [13]:
# df_cz = pd.read_csv('raw/cz_county.csv')
# df_cz = df_cz[~df_cz['County Name'].str.contains('Total')]
# df_cz = df_cz[['LMA/CZ', 'FIPS', 'County Name']]
# df_cz['County Name'] = df_cz['County Name'].str.replace('"', '').str.rstrip(',')
# df_cz['LMA/CZ'] = df_cz['LMA/CZ'].astype(str).str[:-2].astype(int)
# df_cz.to_csv('raw/cz_mappings.csv', index=False)